# CSE-CIC-IDS 2017 Naive Bayes

In [1]:
model_id = "naivebayes1-gaussiannb"

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

import glob

In [3]:
NOTEBOOK_PATH = "C:/Users/Xetrov/Desktop/SciFair20/Code/"

In [4]:
x_scaled = pd.read_csv(NOTEBOOK_PATH + "IDS2017/x_scaled_powertransform.csv")

y_df_enc = pd.read_csv(NOTEBOOK_PATH + "IDS2017/y_grouped_1henc.csv")

# Split data into train and test

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
x_train, x_valtest, y_train, y_valtest = train_test_split(x_scaled, y_df_enc, test_size = 0.4, random_state = 42)

In [7]:
x_val, x_test, y_val, y_test = train_test_split(x_valtest, y_valtest, test_size = 0.5, random_state = 42)

In [8]:
del x_valtest 
del y_valtest

In [9]:
print("Train:")
print(y_train.sum(axis=0))
print("\nValidation:")
print(y_val.sum(axis=0))
print("\nTest:")
print(y_test.sum(axis=0))

Train:
BENIGN          1364100
Botnet             1185
Brute Force        8273
DoS/DDoS         228292
Infiltration         24
PortScan          95267
Web Attack         1304
dtype: int64

Validation:
BENIGN          454207
Botnet             397
Brute Force       2810
DoS/DDoS         76510
Infiltration         7
PortScan         31787
Web Attack         431
dtype: int64

Test:
BENIGN          454790
Botnet             384
Brute Force       2752
DoS/DDoS         75897
Infiltration         5
PortScan         31876
Web Attack         445
dtype: int64


# ADASYN

In [10]:
x_train_res = pd.read_csv(NOTEBOOK_PATH + "IDS2017/x_adasyn.csv").to_numpy()

In [11]:
y_train_res = pd.read_csv(NOTEBOOK_PATH + "IDS2017/y_adasyn.csv")

y_train_res2 = pd.get_dummies(y_train_res[y_train_res.columns[0]])

# Train model

In [26]:
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB, ComplementNB

In [27]:
%%time
clf = ComplementNB().fit(x_train_res, y_train_res)

C:\Users\Xetrov\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


ValueError: Input X must be non-negative

**Save model**

In [14]:
import pickle

In [15]:
import time

model_filename = model_id  + " [" + time.strftime("%Y%m%d %H%M") + "]"

In [16]:
save_file = open(NOTEBOOK_PATH + "Models/" + model_filename + ".pkl", "wb")
saved_model = pickle.dump(clf, save_file)
save_file.close()

**Test model**

In [17]:
pred = clf.predict(x_val)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   26.9s finished


In [18]:
pred_index = list(pred.argmax(1))
y_index = list(np.argmax(y_val.to_numpy(), axis=1))

atktypes = sorted(y_val.columns)
pred_series = pd.Series(pred_index, name="Pred").replace(dict(enumerate(atktypes)))
y_series = pd.Series(y_index, name="Actual").replace(dict(enumerate(atktypes)))

matrix = pd.crosstab(pred_series, y_series, margins=True)
matrix

Actual,BENIGN,Botnet,Brute Force,DoS/DDoS,Infiltration,PortScan,Web Attack,All
Pred,,,,,,,,
BENIGN,453692,24,0,26,0,97,14,453853
Botnet,173,373,0,0,0,0,0,546
Brute Force,0,0,2810,0,0,0,0,2810
DoS/DDoS,150,0,0,76473,0,2,6,76631
Infiltration,0,0,0,0,7,0,0,7
PortScan,191,0,0,11,0,31688,20,31910
Web Attack,1,0,0,0,0,0,391,392
All,454207,397,2810,76510,7,31787,431,566149


# Test Set

In [15]:
from sklearn.metrics import f1_score

In [17]:
pred.shape

(566149,)

In [23]:
pred = clf.predict(x_test)
pred_index = pred
y_index = list(np.argmax(y_test.to_numpy(), axis=1))
f1_micro = f1_score(y_index, pred_index, average='micro')
f1_macro = f1_score(y_index, pred_index, average='macro')

print(f1_micro)
print(f1_macro)

0.5144511427203793
0.23757896640421614


In [24]:
pred = clf.predict(x_test)
pred_index = pred
y_index = list(np.argmax(y_test.to_numpy(), axis=1))

atktypes = sorted(y_val.columns)
pred_series = pd.Series(pred_index, name="Pred").replace(dict(enumerate(atktypes)))
y_series = pd.Series(y_index, name="Actual").replace(dict(enumerate(atktypes)))

matrix = pd.crosstab(pred_series, y_series, margins=True)
# matrix.to_csv(NOTEBOOK_PATH + "Confusion Matrices/test_" + model_filename + ".csv")
matrix

Actual,BENIGN,Botnet,Brute Force,DoS/DDoS,Infiltration,PortScan,Web Attack,All
Pred,,,,,,,,
BENIGN,243819,116,0,34423,1,9,0,278368
Botnet,25618,241,0,0,0,0,0,25859
Brute Force,25352,0,1358,12230,0,81,33,39054
DoS/DDoS,57081,0,3,14213,0,180,375,71852
Infiltration,61293,27,0,1693,4,2,0,63019
PortScan,24295,0,0,2,0,31584,0,55881
Web Attack,17332,0,1391,13336,0,20,37,32116
All,454790,384,2752,75897,5,31876,445,566149


# Log results

In [22]:
model_log = open(NOTEBOOK_PATH + "model_log.txt", "a")

model_log.write("\n" + model_filename)
model_log.write("\n\tF1 Micro: " + str(f1_micro))
model_log.write("\n\tF1 Macro: " + str(f1_macro))

model_log.close()